In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import time
from MMD_metric import _mix_rbf_kernel, mix_rbf_mmd2, _mmd2
from CCF import CCF

# set the tensorflow verbosity
tf.logging.set_verbosity(tf.logging.DEBUG)

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 

C:\Users\user\Anaconda3\envs\tf_gpu_1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\envs\tf_gpu_1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\envs\tf_gpu_1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\envs\tf_gpu_1.14\lib\site-packages

In [2]:
index = 'tep11'

folder = 'AE_BP_' + index + '_CCF_final_no_ccf/'

model_ = 'model_' + index
model = model_+'.ckpt' #'model_index_select.ckpt'

if not os.path.isdir(folder):
    os.mkdir(folder)
    
saver_path = os.path.join(folder, model)

restore_path = os.path.join(folder)

In [3]:
data_index = "d11_te.dat"
data_test_index = "d11.dat"

data_folder = "tennessee-eastman-profBraatz-master/"
data_path = data_folder + data_index
data_test_path = data_folder + data_test_index


dat = np.genfromtxt(data_path)
tep_data = dat[150:450]
print(tep_data.shape)


dat_test = np.genfromtxt(data_test_path)
tep_data_test = dat_test[200:400]
print(tep_data_test.shape)

(200, 52)
(200, 52)


In [6]:
tep_data_pd = pd.DataFrame(tep_data)
CCFtrain=CCF(tep_data_pd,maxlag=0)
CCFtrain.cal()
# CCFtrain.show()
tep_data_matrix = CCFtrain.get_matrix()

In [7]:
tep_data_matrix = np.array(tep_data_matrix)
ccf_value = 0.4

temp = 0
for i in range(len(tep_data_matrix)):
    for j in range(len(tep_data_matrix)):
        if tep_data_matrix[i,j] >= ccf_value:
            temp = temp + 1
dim_ccf = int((temp-52)/2)
dim_ccf_ratio = dim_ccf/((51+1)*51/2)

print(dim_ccf)
print(dim_ccf_ratio)

41
0.03092006033182504


In [8]:
data_mean = np.mean(tep_data,0)   #(72,)
data_std = np.std(tep_data,0,ddof=1) 

tep_data = (tep_data - data_mean)/data_std
tep_data_test = (tep_data_test - data_mean)/data_std

print(tep_data.shape)
print(tep_data_test.shape)

(200, 52)
(200, 52)


In [9]:
def add_window_multiply(time_series, time_step, multiply):
    
    series_window_multiply = []
    L = len(time_series)

    start = 0
    end = 0
    for i in range(L//time_step):
        end = end+time_step
        dat = time_series[start:end]
        for j in range(multiply):
            series_window_multiply.append(dat)
        start = end
    
    series_window_multiply = np.array(series_window_multiply)
    
    return series_window_multiply

In [10]:
nsteps_gan = 1   
multiply = 1     

tep_data_win = add_window_multiply(tep_data, nsteps_gan, multiply)
print("tep_data_win shape:",tep_data_win.shape)

tep_data_win shape: (200, 1, 52)


In [11]:
# training parameters
batch_size = 10

dim_input = tep_data_win.shape[-1] 
timestep = tep_data_win.shape[1]
dim_z = int(dim_input*(1-dim_ccf_ratio))
print(dim_z)

# variables : input
X = tf.placeholder(tf.float32, shape=(None, timestep, dim_input))
Z = tf.placeholder(tf.float32, shape=(None, timestep, dim_z))
keep_prob_feed = tf.placeholder(tf.float32)
global_step = tf.Variable(0, trainable = False)
batch_size_feed = tf.placeholder(tf.float32)

50


In [12]:
def bp_encoder(X, keep_prob_feed, batch_size_feed):
    
    X_in = tf.reshape(X, (batch_size_feed, timestep, dim_input))
    
    with tf.variable_scope("bp_encoder", reuse = tf.AUTO_REUSE):

        hidden = tf.layers.dense(X_in, units = 256, activation=tf.nn.relu)
        hidden = tf.layers.dropout(hidden, rate = 1-keep_prob_feed) 
        
        hidden = tf.layers.dense(hidden, units = 512, activation=tf.nn.relu)
        hidden = tf.layers.dropout(hidden, rate = 1-keep_prob_feed)
        
        hidden = tf.layers.dense(hidden, units = 128, activation=tf.nn.relu)
        hidden = tf.layers.dropout(hidden, rate = 1-keep_prob_feed) 
        
        hidden = tf.layers.dense(hidden, units = dim_z)
        hidden = tf.layers.dropout(hidden, rate = 1-keep_prob_feed)  #(batch_size, timestep, dim_z)

        z = tf.reshape(hidden, (batch_size_feed, timestep, dim_z))
                
    return z

def bp_decoder(Z, keep_prob_feed, batch_size_feed):
    
    Z_in = tf.reshape(Z, (batch_size_feed, timestep, dim_z))

    with tf.variable_scope("bp_decoder", reuse = tf.AUTO_REUSE):

        hidden = tf.layers.dense(Z_in, units = 128, activation=tf.nn.relu) 
        hidden = tf.layers.dropout(hidden, rate = 1-keep_prob_feed) 
        
        hidden = tf.layers.dense(hidden, units = 512, activation=tf.nn.relu)
        hidden = tf.layers.dropout(hidden, rate = 1-keep_prob_feed)
        
        hidden = tf.layers.dense(hidden, units = 256, activation=tf.nn.relu)
        hidden = tf.layers.dropout(hidden, rate = 1-keep_prob_feed) 
        
        hidden = tf.layers.dense(hidden, units = dim_input)
        hidden = tf.layers.dropout(hidden, rate = 1-keep_prob_feed)
        
        gen_outputs = tf.reshape(hidden, (batch_size_feed, timestep, dim_input))

        return gen_outputs
    
def VAE_BP(X, keep_prob_feed, batch_size_feed):
    z = bp_encoder(X, keep_prob_feed, batch_size_feed)
    generated_output = bp_decoder(z, keep_prob_feed, batch_size_feed)

    return X, z, generated_output

In [13]:
def CCF_tf_two(x,y):

    x_shape = tf.shape_n([x])
    len_x = x_shape[0][0]
    len_x = tf.cast(len_x, tf.float32)
    
    sum_x = tf.reduce_sum(x, 0)   
    sum_y = tf.reduce_sum(y, 0) 
    
    sum_xy = tf.reduce_sum(tf.multiply(x,y))

    sum_xx = tf.reduce_sum(tf.multiply(x,x))
    sum_yy = tf.reduce_sum(tf.multiply(y,y))
    
    num = sum_xy-(tf.multiply(sum_x, sum_y) / len_x)
    
    den_x = (sum_xx)-(tf.multiply(sum_x, sum_x)/len_x)
    den_y = (sum_yy)-(tf.multiply(sum_y, sum_y)/len_x)
    den = tf.multiply(den_x, den_y)
    den = tf.sqrt(den)
    
    ccf = tf.divide(num, den)
    return ccf

In [14]:
def CCF_tf(latent_data, batch_size_feed):
    latent_data = tf.reshape(latent_data, (batch_size_feed*timestep, dim_z))
    
    unit_ccf = tf.Variable([])
    
    for i in range(0, dim_z):
        for j in range(dim_z-1, i, -1):
            x = latent_data[:, i]
            x = tf.cast(x, tf.float32)
            y = latent_data[:, j]
            y = tf.cast(y, tf.float32)
            temp = CCF_tf_two(x,y)
            temp = tf.abs(temp, name=None) 
            unit_ccf = tf.concat([unit_ccf, [temp]], 0)
          
    sum_ccf = tf.reduce_sum(unit_ccf) 
    max_ccf = tf.reduce_max(unit_ccf)
    return sum_ccf, unit_ccf, max_ccf

In [15]:
lr = 0.0001
decay_rate = 0.8
decay_step = 400

# MODEL
real_data, latent_data, fake_data = VAE_BP(X, keep_prob_feed, batch_size_feed)

Recon_error = tf.losses.mean_squared_error(real_data, fake_data)

total_error = Recon_error 

learning_rate_decayed = lr*decay_rate**(global_step/decay_step)
optimizer = tf.train.AdamOptimizer(learning_rate_decayed).minimize(total_error)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
train_epoch = 400  #300
keep_prob = 0.0001
batch_size_train = batch_size

# save loss
train_hist = {}
train_hist['Recon_e_list'] = []
train_hist['Total_Error_list'] = []
train_hist['sum_CCF'] = []

valid_hist = {}
valid_hist['Recon_e_list_valid'] = []

saver = tf.train.Saver()

session_config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth=True))
with tf.Session(config = session_config) as sess:
    sess.run(tf.global_variables_initializer())

    print('Optimization start!')
    start = time.time()
    
    for epoch in range(train_epoch):
        Recon_e_list = []
        Total_Error_list = []
        ori_data = []
        gen_data = []
        sum_CCF = []
        
        for i in range(len(tep_data_win) // batch_size): # 250/5
            real_data = tep_data_win[i * batch_size: (i + 1) * batch_size]     #(batch_size)
            real_data = real_data.reshape((batch_size, timestep, dim_input))

            _, total_error_, Recon_e = sess.run([optimizer, total_error, Recon_error], 
                                                    feed_dict = {X: real_data, 
                                                                 keep_prob_feed: keep_prob,
                                                                 global_step: epoch,
                                                                 batch_size_feed: batch_size_train
                                                                })
            
            generated_output = sess.run([fake_data, ], 
                                feed_dict = {X: real_data, 
                                             keep_prob_feed: keep_prob,
                                             global_step: epoch,
                                             batch_size_feed: batch_size_train
                                            })     
            
            Recon_e_list.append(Recon_e)
            Total_Error_list.append(total_error_)
            ori_data.append(real_data)   
            gen_data.append(generated_output)  
            

        ''' valid '''
        ori_data_test2 = []
        gen_data_test2 = []    
        batch_size_test2 = len(tep_data_test)

        real_data = tep_data_test
        real_data = real_data.reshape((-1, timestep, dim_input))

        generated_output_test2, Recon_error_valid = sess.run([fake_data, Recon_error], 
                            feed_dict = {X: real_data, 
                                         keep_prob_feed: 1,
                                         global_step: epoch,
                                         batch_size_feed: batch_size_test2
                                        })    

        ori_data_test2.append( real_data.reshape((-1, dim_input)) )
        gen_data_test2.append( np.array(generated_output_test2).reshape((-1, dim_input))) 
        valid_hist['Recon_e_list_valid'].append(Recon_error_valid)  
        
    ############        show result      #######################
        if (epoch+1) % 1 == 0:
            print('[%d/%d] Total_Error: %.3f, loss_Recon: %.3f'%((epoch + 1), train_epoch,
                                                np.mean(Total_Error_list), np.mean(Recon_e_list)))
            
            print('VALID loss_Recon: %.3f'%(Recon_error_valid))
        train_hist['Recon_e_list'].append(np.mean(Recon_e_list))  
        train_hist['Total_Error_list'].append(np.mean(Total_Error_list))  

    #############        save model      #######################
    saver.save(sess, saver_path)   
    sess.close()
    
    
print("Optimization Finished!")
end = time.time()
print("runtime：%f 秒" % (end - start))

Optimization start!
[1/400] Total_Error: 0.999, loss_Recon: 0.999
VALID loss_Recon: 1.225
[2/400] Total_Error: 0.981, loss_Recon: 0.981
VALID loss_Recon: 1.217
[3/400] Total_Error: 0.966, loss_Recon: 0.966
VALID loss_Recon: 1.203
[4/400] Total_Error: 0.944, loss_Recon: 0.944
VALID loss_Recon: 1.175
[5/400] Total_Error: 0.906, loss_Recon: 0.906
VALID loss_Recon: 1.119
[6/400] Total_Error: 0.843, loss_Recon: 0.843
VALID loss_Recon: 1.025
[7/400] Total_Error: 0.757, loss_Recon: 0.757
VALID loss_Recon: 0.934
[8/400] Total_Error: 0.678, loss_Recon: 0.678
VALID loss_Recon: 0.873
[9/400] Total_Error: 0.610, loss_Recon: 0.610
VALID loss_Recon: 0.819
[10/400] Total_Error: 0.554, loss_Recon: 0.554
VALID loss_Recon: 0.772
[11/400] Total_Error: 0.509, loss_Recon: 0.509
VALID loss_Recon: 0.738
[12/400] Total_Error: 0.468, loss_Recon: 0.468
VALID loss_Recon: 0.719
[13/400] Total_Error: 0.436, loss_Recon: 0.436
VALID loss_Recon: 0.708
[14/400] Total_Error: 0.415, loss_Recon: 0.415
VALID loss_Recon: 0

KeyboardInterrupt: 

In [ ]:
plt.figure(0)  
plt.plot(train_hist['Recon_e_list'])

In [ ]:
plt.plot(valid_hist['Recon_e_list_valid'])

In [ ]:
ori_data = np.array(ori_data).reshape((-1, dim_input))
gen_data = np.array(gen_data).reshape((-1, dim_input))

ori_data = (ori_data*data_std) + data_mean
gen_data = (gen_data*data_std) + data_mean

print(ori_data.shape)
print(gen_data.shape)

In [ ]:
for i in range(tep_data.shape[1]):
    plt.figure(i)  
    plt.title(i)
    plt.plot(ori_data[:,i],'r')
    plt.plot(gen_data[:,i])
